# 如何向子图添加线程级持久性

<div class="警告提示">
<p class="admonition-title">先决条件</p>
<p>
本指南假设您熟悉以下内容：
<ul>
<li>
<a href="https://langchain-ai.github.io/langgraphjs/concepts/low_level/#subgraphs">
子图
</a>
</li>
<li>
<a href="https://langchain-ai.github.io/langgraphjs/concepts/persistence/">
坚持
</a>
</li>
</ul>
</p>
</div>

本指南展示了如何将[线程级](https://langchain-ai.github.io/langgraphjs/how-tos/persistence/)持久性添加到使用[子图](https://langchain-ai.github.io/langgraphjs/how-tos/subgraph/)的图表中。

## 设置

首先，让我们安装所需的软件包：
```bash
$ npm install @langchain/langgraph @langchain/core
```

<div class="警告提示">
<p class="admonition-title">设置 <a href="https://smith.langchain.com">LangSmith</a> 进行 LangGraph 开发</p>
<p style="padding-top: 5px;">
注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 - 在<a href="https://docs.smith.langchain.com">此处</a>了解有关如何开始的更多信息。
</p>
</div>

## 定义具有持久性的图

要为具有子图的图添加持久性，您需要做的就是在**编译父图**时传递一个检查点（https://langchain-ai.github.io/langgraphjs/reference/classes/checkpoint.BaseCheckpointSaver.html）。LangGraph 会自动将检查指针传播到子子图。

<div class="警告说明">
<p class="admonition-title">注意</p>
<p>
编译子图时<b>不应该提供</b>检查点。相反，您必须定义一个传递给 <code>parentGraph.compile()</code> 的**单个**检查点，LangGraph 会自动将该检查点传播到子子图。如果将检查指针传递给 <code>subgraph.compile()</code>，它将被忽略。当您<a href="../subgraph#add-a-node-function-that-invokes-the-subgraph">添加显式调用子图的节点</a>时，这也适用。
</p>
</div>

让我们定义一个带有单个子图节点的简单图来展示如何执行此操作。

In [1]:
import { StateGraph, Annotation } from "@langchain/langgraph";

// 子图

const SubgraphStateAnnotation = Annotation.Root({
  foo: Annotation<string>,
  bar: Annotation<string>,
});

const subgraphNode1 = async (state: typeof SubgraphStateAnnotation.State) => {
  return { bar: "bar" };
};

const subgraphNode2 = async (state: typeof SubgraphStateAnnotation.State) => {
  // 请注意，该节点使用的状态键（“bar”）仅在子图中可用
  // 并正在发送共享状态密钥（'foo'）的更新
  return { foo: state.foo + state.bar };
};

const subgraph = new StateGraph(SubgraphStateAnnotation)
  .addNode("subgraphNode1", subgraphNode1)
  .addNode("subgraphNode2", subgraphNode2)
  .addEdge("__start__", "subgraphNode1")
  .addEdge("subgraphNode1", "subgraphNode2")
  .compile();
  
// 父图
const StateAnnotation = Annotation.Root({
  foo: Annotation<string>,
});

const node1 = async (state: typeof StateAnnotation.State) => {
  return {
    foo: "hi! " + state.foo,
  };
};

const builder = new StateGraph(StateAnnotation)
  .addNode("node1", node1)
  // 请注意，我们将编译的子图作为节点添加到父图
  .addNode("node2", subgraph)
  .addEdge("__start__", "node1")
  .addEdge("node1", "node2");

我们现在可以使用内存检查指针（`MemorySaver`）来编译该图。

In [2]:
import { MemorySaver } from "@langchain/langgraph-checkpoint";

const checkpointer = new MemorySaver();

// 编译父图时只能通过检查点。
// LangGraph 会自动将检查指针传播到子子图。

const graph = builder.compile({
  checkpointer: checkpointer
});

## 验证持久性是否有效

现在让我们运行该图并检查父图和子图的持久化状态以验证持久化是否有效。我们应该期望在 `state.values` 中看到父图和子图的最终执行结果。

In [3]:
const config = { configurable: { thread_id: "1" } };

In [5]:
const stream = await graph.stream({
  foo: "foo"
}, {
  ...config,
  subgraphs: true,
});

for await (const [_source, chunk] of stream) {
  console.log(chunk);
}

{ node1: { foo: 'hi! foo' } }
{ subgraphNode1: { bar: 'bar' } }
{ subgraphNode2: { foo: 'hi! foobar' } }
{ node2: { foo: 'hi! foobar' } }


现在，我们可以通过使用与调用图形相同的配置调用 `graph.get_state()` 来查看父图形状态。

In [6]:
(await graph.getState(config)).values;

{ foo: 'hi! foobar' }


要查看子图状态，我们需要做两件事：

1. 查找子图的最新配置值
2. 使用 `graph.getState()` 检索最新子图配置的值。

为了找到正确的配置，我们可以检查父图的状态历史记录，并在从 `node2`（带有子图的节点）返回结果之前找到状态快照：

In [9]:
let stateWithSubgraph;

const graphHistories = await graph.getStateHistory(config);

for await (const state of graphHistories) {
  if (state.next[0] === "node2") {
    stateWithSubgraph = state;
    break;
  }
}

状态快照将包含接下来要执行的 `tasks` 列表。使用子图时，`tasks` 将包含我们可用于检索子图状态的配置：

In [10]:
const subgraphConfig = stateWithSubgraph.tasks[0].state;

console.log(subgraphConfig);

{
  configurable: {
    thread_id: '1',
    checkpoint_ns: 'node2:25814e09-45f0-5b70-a5b4-23b869d582c2'
  }
}


In [11]:
(await graph.getState(subgraphConfig)).values

{ foo: 'hi! foobar', bar: 'bar' }


如果您想了解有关如何修改人机循环工作流程的子图状态的更多信息，请查看此[操作指南](https://langchain-ai.github.io/langgraph/how-tos/subgraphs-manage-state/)。